---

<!-- # **Profissão: Cientista de Dados** -->
### Combinação de modelos I | Exercício 2

**Aluno:** Guilherme Rhein<br>

---

# Tarefa 02

**1. Monte um passo a passo para o algoritmo RF:**

> O **Ramdom Forest** é um procedimento procedimento composto pelos seguintes passos:
> 1. **Bootstrap com Feature Selection:** No processo de bootstrap, são geradas amostras aleatórias de treinamento com reposição a partir do conjunto de dados original. Cada amostra bootstrap mantém o mesmo tamanho do conjunto de dados original em números de linhas e permite que cada exemplo seja selecionado várias vezes. Esse método visa introduzir variabilidade nos conjuntos de dados utilizados para treinar modelos subsequentes, porém respeitando uma regra definida para o número de colunas em cada amostra. Quando se tratar de classificação é determinado o uso da raiz quadrada do número total de variáveis na base, já para regressão, um terço das variáveis na base. Assim tentamos evitar Overfitting<br>

> 2. **Modelagem** Para cada amostra Bagging gerada, um modelo de Machine Learning é treinado. Isso implica na criação de vários modelos, onde cada um é treinado em uma amostra com resultados individuais.<br>

> 4. **Aggregating:** Os resultados dos modelos individuais são avaliados para que seja dado uma previsão final com base em cada resultado de cada modelo. Quando estamos lidando com questões de classificação, a previsão final será dada pela moda dos resultados dos modelos. Quando os dados são de regressão, a previsão final será a média dos resultados dos modelos.

<br>**2. Explique com suas palavras o Random forest:**

> O ***Random Forest*** é uma técnica muito próxima do ***Bootstrap Aggregating*** que combina várias instâncias de um mesmo modelo de aprendizado de máquina para melhorar a precisão e robustez do modelo final, porém com limite de dados para evitar Overfitting:
> 1. ***Bootstrap com Feature Selection:*** Para criar diferentes conjuntos de treinamento, o Bagging utiliza amostragem bootstrap. Isso significa que para cada árvore no RandomForest, são amostrados aleatoriamente exemplos do conjunto de treinamento com substituição, o que significa que um exemplo pode ser escolhido mais de uma vez. O diferencial está na determinação do ***Feature Selection*** respeitando uma regra definida para o número de colunas em cada amostra utilizada para o treinamento da árvore. Quando os dados forem de classificação é determinado o número de colunas das amostras sendo a raiz quadrada do número total de variáveis na base, já para regressão está definido um terço das variáveis na base. Assim tentamos evitar Overfitting<br>

> 2. ***Construção de Árvores:*** Para cada conjunto de treinamento amostrado, uma árvore de decisão é construída. Cada árvore é treinada de forma independente e pode aprender padrões diferentes nos dados.<br>

> 4. ***Agregação:*** Após a construção de todas as árvores, a predição final é feita agregando as previsões de cada árvore. No caso de classificação, isso geralmente envolve uma votação majoritária, enquanto em regressão, pode envolver a média das previsões.
>
>A principal ideia por trás do ***Random Forest*** é reduzir a variância do modelo, tornando-o mais robusto ao reduzir a sensibilidade a padrões específicos nos dados de treinamento. Isso é particularmente eficaz quando o modelo é propenso a overfitting, melhorando a generalização e desempenho do modelo.

<br>**3. Qual a diferença entre Bagging e Random Forest?**

Bagging e Random Forest diferem no desempenho e na redução da variância. Bagging combina modelos usando amostragem aleatória, enquanto Random Forest é uma variação específica que utiliza árvores de decisão. No Random Forest, cada modelo é treinado em subconjuntos aleatórios do conjunto de dados original, aumentando a robustez e a precisão das previsões em comparação com o Bagging. Assim, Random Forest é uma extensão aprimorada do Bagging, oferecendo desempenho superior e uma redução adicional na variância dos resultados.

In [181]:
import pandas as pd

from sklearn.datasets import load_diabetes

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

<br>**Aplicando o processo:**

In [182]:
df = pd.DataFrame(data=load_diabetes().data, columns=load_diabetes().feature_names)
df['target'] = load_diabetes().target
df_train, df_test = train_test_split(df, test_size = 0.30)

X_test = df_test.drop(['target'], axis=1)
y_test = df_test[['target']]

y_pred_bagging = {}
for i in range(10):
    df_train = df_train.sample(n=len(df_train), replace=True)
    X_train = df_train.drop(['target'], axis=1)
    X_train = X_train.sample(n=round((df_train.shape[1]/3)-1), axis=1)
    y_train = df_train[['target']]

    clf = DecisionTreeRegressor()
    clf.fit(X_train, y_train)

    y_pred_bagging.update({i:clf.predict(X_test[X_train.columns])})

predict = pd.DataFrame(y_pred_bagging).mean(axis=1).rename('y_pred')

C:\Users\Guilherme Rhein\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\Guilherme Rhein\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Guilherme Rhein\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\Guilherme Rhein\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinsta

In [183]:
pd.concat(objs=[y_test.reset_index(drop=True), predict], axis=1)

,target,y_pred
0,272.0,168.3
1,242.0,198.0
2,230.0,222.7
3,72.0,135.3
4,113.0,113.0
...,...,...
128,84.0,125.1
129,97.0,134.0
130,139.0,182.1
131,215.0,180.4


In [184]:
print('Mean squared error:', mean_squared_error(y_test, predict))
print('Coefficient of determination:', r2_score(y_test, predict))

Mean squared error: 4627.366992481204
Coefficient of determination: 0.27463919976260565


C:\Users\Guilherme Rhein\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\Guilherme Rhein\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Guilherme Rhein\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\Guilherme Rhein\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinsta